### feature extraction

In [3]:
from feature_extractor import *
from data_preprocess import *
from common.utils import ImageSlice, ImageSliceInMemory
import os
import gc
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
class_feature_extrctor = FeatureExtrctor(gpu_num = 4, gpus = "0,1,2,3")
class_feature_extrctor.build_model()

input_2
lambda_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
drop_fc1
fc2
drop_fc2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooli

In [5]:
features = []
err = 0

# predict img pathes
# data_dir = "/home/sakulaki/TCTDATA/gz_new_unlabeled/2018-06-11-normal/"
data_dir = "/home/sakulaki/TCTDATA_test/20180813/"
images_origin = os.listdir(data_dir)

for i, image in enumerate(images_origin):
    err = class_feature_extrctor.read_big_pic(data_dir + image)
    if (0 == err):
        feature = class_feature_extrctor.feature_extrctor()
        features.append(feature)
    else:
        # error, do nothing
        pass

features = np.array(features)

del class_feature_extrctor
gc.collect()

/home/sakulaki/TCTDATA_test/20180813/2018-08-13-15_02_29.kfb
Image Process /home/sakulaki/TCTDATA_test/20180813/2018-08-13-15_02_29.kfb ...
Adding Job to Pool...
28672 28672
Done, cost: 0:00:00.888169, Total Job Count: 128, Worker Count: 26
One Job Done, Got 128 patches, last Job Count: 127
One Job Done, Got 128 patches, last Job Count: 126
One Job Done, Got 128 patches, last Job Count: 125
One Job Done, Got 128 patches, last Job Count: 124
One Job Done, Got 128 patches, last Job Count: 123
One Job Done, Got 128 patches, last Job Count: 122
One Job Done, Got 128 patches, last Job Count: 121
One Job Done, Got 128 patches, last Job Count: 120
One Job Done, Got 128 patches, last Job Count: 119
One Job Done, Got 128 patches, last Job Count: 118
One Job Done, Got 128 patches, last Job Count: 117
One Job Done, Got 128 patches, last Job Count: 116
One Job Done, Got 128 patches, last Job Count: 115
One Job Done, Got 128 patches, last Job Count: 114
One Job Done, Got 128 patches, last Job Count

KeyboardInterrupt: 

In [4]:
import h5py
h5f = h5py.File('./features_test.h5', 'w')
h5f.create_dataset('features', data=features)
h5f.close()

### diagnosis

In [6]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.applications.vgg19 import preprocess_input
from keras.utils.training_utils import multi_gpu_model
from keras.utils import to_categorical
import tensorflow as tf
import cv2
import os
import random
import h5py
import numpy as np

In [7]:
h5f = h5py.File('./features_test.h5','r')
features = h5f['features'][:]
h5f.close()
print("du", features.shape)

du (39, 128, 128, 512)


In [13]:
with tf.device('/cpu:0'):
    input_tensor = Input((128, 128, 512))
    
    conv1_1 = Conv2D(filters = 1024, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_1')(input_tensor)
    conv1_2 = Conv2D(filters = 1024, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_2')(conv1_1)
    mp1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv1_2)
    
    conv2_1 = Conv2D(filters = 2048, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_1')(mp1)
    conv2_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_2')(conv2_1)
    mp2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv2_2)
    
    conv3_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_1')(mp2)
    conv3_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_2')(conv3_1)
    mp3 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv3_2)
    
    conv4_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_1')(mp3)
    conv4_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_2')(conv4_1)
    mp4 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv4_2)
    
    conv5_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv5_1')(mp4)
    conv5_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv5_2')(conv5_1)

    gmp = GlobalMaxPooling2D()(conv5_2)
    drop = Dropout(0.5, name='drop_fc1')(gmp)
    predictions = Dense(2, activation='softmax')(drop)

    model = Model(inputs=input_tensor, outputs=predictions)
    model.load_weights('diagnosis.h5')
    model.summary()

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.summary()

result = parallel_model.predict(features, batch_size=4, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 128, 128, 512)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 128, 128, 1024)    4719616   
_________________________________________________________________
conv1_2 (Conv2D)             (None, 128, 128, 1024)    1049600   
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 64, 64, 1024)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 64, 64, 2048)      18876416  
_________________________________________________________________
conv2_2 (Conv2D)             (None, 64, 64, 2048)      4196352   
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 32, 32, 2048)      0         
__________

### evaluate

In [14]:
class_name = ['Normal', 'Abnormal']
for i, r in enumerate(result):
    print("du", images_origin[i], ":", class_name[np.argmax(r)])

du 2018-08-13-15_02_29.kfb : Normal
du 2018-08-13-14_55_37.kfb : Abnormal
du 2018-08-13-15_14_51.kfb : Abnormal
du 2018-08-13-15_18_04.kfb : Abnormal
du 2018-08-13-14_47_18.kfb : Abnormal
du 2018-08-13-14_24_38.kfb : Normal
du 2018-08-13-14_41_38.kfb : Normal
du 2018-08-13-14_19_27.kfb : Normal
du 2018-08-13-15_09_48.kfb : Abnormal
du 2018-08-13-14_29_33.kfb : Abnormal
du 2018-08-13-14_22_54.kfb : Abnormal
du 2018-08-13-14_31_19.kfb : Abnormal
du 2018-08-13-15_16_28.kfb : Abnormal
du 2018-08-13-14_33_05.kfb : Normal
du 2018-08-13-15_24_52.kfb : Abnormal
du 2018-08-13-15_04_07.kfb : Normal
du 2018-08-13-15_21_29.kfb : Abnormal
du 2018-08-13-14_59_06.kfb : Abnormal
du 2018-08-13-14_52_19.kfb : Normal
du 2018-08-13-14_54_02.kfb : Abnormal
du 2018-08-13-14_48_56.kfb : Normal
du 2018-08-13-15_19_47.kfb : Normal
du 2018-08-13-15_11_25.kfb : Normal
du 2018-08-13-14_45_38.kfb : Abnormal
du 2018-08-13-14_21_10.kfb : Abnormal
du 2018-08-13-14_50_37.kfb : Abnormal
du 2018-08-13-14_43_21.kfb : Nor